In [2]:
from scipy.spatial.distance import pdist, cdist, squareform
import math
import pandas as pd
import numpy as np
from numba import cuda
from numba import jit
from sklearn import datasets
import cupy as cp

In [40]:
a = np.array([[1, 1], [2, 2], [3, 3]])
b = np.array([[5,5,5]]).reshape(-1,1)

a = cp.asarray(a)
b = cp.asarray(b)
c = cp.concatenate((b,a), axis = 1)
c = cp.asnumpy(c)

print(c)
print(c.shape)

[[1 1]
 [2 2]
 [3 3]]
(3, 1)


In [43]:
d = np.concatenate((b,a), axis = 1)
print(d)

[[5 1 1]
 [5 2 2]
 [5 3 3]]


In [108]:
def argwhere(data):
    aux1 = cp.nonzero(data)
    aux2 = aux1[0].reshape(-1,1)
    
    for i in range (1,len(aux1)):
        aux2 = cp.concatenate((aux1[i].reshape(-1,1),aux2), axis = 1)
    
    return aux2

In [246]:
x = np.array([[1,2,3],[4,5,6],[4,5,6],[4,5,6]])
print(x)

z = cp.asarray(x)
print(z)

[[1 2 3]
 [4 5 6]
 [4 5 6]
 [4 5 6]]
[[1 2 3]
 [4 5 6]
 [4 5 6]
 [4 5 6]]


In [247]:
a = np.argwhere(x)
print(a)

a = argwhere(z)
print(a)

[[0 0]
 [0 1]
 [0 2]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]
 [2 2]
 [3 0]
 [3 1]
 [3 2]]
[[0 0]
 [1 0]
 [2 0]
 [0 1]
 [1 1]
 [2 1]
 [0 2]
 [1 2]
 [2 2]
 [0 3]
 [1 3]
 [2 3]]


In [248]:
b = np.argwhere(np.isnan(x))
print(b)

b = argwhere(cp.isnan(z))
print(b)

[]
[]


In [23]:
Uniquesample = np.random.rand(2,8)

UN, W = Uniquesample.shape

Xnorm = np.matrix(np.sqrt(np.sum(np.power(Uniquesample,2),axis=1))).T
aux2 = Xnorm
for i in range(W-1):
    aux2 = np.insert(aux2,0,Xnorm.T,axis=1)
Uniquesample1 = Uniquesample / aux2
AA2 = np.mean(Uniquesample1,0)
X2 = 1
DT2 = X2 - np.sum(np.power(AA2,2))
aux = []
for i in range(UN): aux.append(AA2)

for i in range(UN):
    print(Uniquesample1[i].shape)
    print(Uniquesample1[i])    
    print('_'*50)
    print(np.shape(aux[i]))
    print(aux[i])
    print('_'*50)
    aux2 = [Uniquesample1[i]-aux[i]]
    print(Uniquesample1[i]-aux[i])
    print(np.shape(aux2))
    print(aux2)
    print('_._'*20)
    
a = np.power(aux2,2)

b = np.sum(a,axis=1)

c = np.sum(b,axis=1)

(1, 8)
[[0.43532162 0.29205007 0.47768188 0.19451897 0.38094426 0.29656143
  0.07485497 0.46958893]]
__________________________________________________
(1, 8)
[[0.3167086  0.15618952 0.48455209 0.21050753 0.26273079 0.17950667
  0.33111275 0.5076309 ]]
__________________________________________________
[[ 0.11861302  0.13586055 -0.00687021 -0.01598857  0.11821346  0.11705476
  -0.25625778 -0.03804197]]
(1, 1, 8)
[matrix([[ 0.11861302,  0.13586055, -0.00687021, -0.01598857,  0.11821346,
          0.11705476, -0.25625778, -0.03804197]])]
_.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__.__._
(1, 8)
[[0.19809558 0.02032896 0.4914223  0.2264961  0.14451733 0.06245191
  0.58737053 0.54567287]]
__________________________________________________
(1, 8)
[[0.3167086  0.15618952 0.48455209 0.21050753 0.26273079 0.17950667
  0.33111275 0.5076309 ]]
__________________________________________________
[[-0.11861302 -0.13586055  0.00687021  0.01598857 -0.11821346 -0.11705476
   0.25625778  0.03

In [ ]:

def pi_calculator(Uniquesample, mode):
    UN, W = Uniquesample.shape
    if mode == 'euclidean' or mode == 'mahalanobis' or mode == 'cityblock' or mode == 'chebyshev' or mode == 'canberra':
        AA1 = Uniquesample.mean(0)
        X1 = sum(sum(np.power(Uniquesample,2)))/UN
        DT1 = X1 - sum(np.power(AA1,2))
        aux = []
        for i in range(UN): aux.append(AA1)
        aux2 = [Uniquesample[i]-aux[i] for i in range(UN)]
        uspi = np.sum(np.power(aux2,2),axis=1)+DT1
        
    if mode == 'minkowski':
        AA1 = Uniquesample.mean(0)
        X1 = sum(sum(np.power(Uniquesample,2)))/UN
        DT1 = X1 - sum(np.power(AA1,2))
        aux = np.matrix(AA1)
        for i in range(UN-1): aux = np.insert(aux,0,AA1,axis=0)
        aux = np.array(aux)
        
        uspi = np.power(cdist(Uniquesample, aux, mode, p=1.5),2)+DT1
        uspi = uspi[:,0]
    
    if mode == 'cosine':
        Xnorm = np.matrix(np.sqrt(np.sum(np.power(Uniquesample,2),axis=1))).T
        aux2 = Xnorm
        for i in range(W-1):
            aux2 = np.insert(aux2,0,Xnorm.T,axis=1)
        Uniquesample1 = Uniquesample / aux2
        AA2 = np.mean(Uniquesample1,0)
        X2 = 1
        DT2 = X2 - np.sum(np.power(AA2,2))
        aux = []
        for i in range(UN): aux.append(AA2)
        aux2 = [Uniquesample1[i]-aux[i] for i in range(UN)]
        uspi = np.sum(np.sum(np.power(aux2,2),axis=1),axis=1)+DT2
        
    return uspi